# PI7 Onderzoeksrapport
#### Dajmen Graus en Bram Verheijen 

# TODO
## Accuracy score by linear regression (werken met train en test set)
## Opdelen code stukken in kleinere stukken
## Tekst schrijven

## Data Selectie/Preperation
Voor het verkrijgen van een passende dataset werden meerdere bronnen doorzocht met verzamelingen van verschillende datasets. Deze datasets werden langs de eisen vanuit de opdrachtomschrijving gelegd, waardoor uiteindlijk een dataset werd gekozen die gepast is voor de opdracht. De gekozen dataset heeft alleen één datatype, namelijk getallen. Echter bleek deze dataset zeer geschikt te zijn voor het gebruik met classification en regression. 

### Gekozen Dataset
De dataset die uiteindelijk werd gekozen voor deze opdracht, is een dataset over de kwaliteit van verschillende rode wijnen. De kwaliteit wordt bepaald op basis van 11 aspecten:
- fixed acidity
- volatile acidity
- citric acid
- residual sugar
- chlorides
- free sulfur dioxide
- total sulfur dioxide
- density
- pH
- sulphates
- alcohol

Uit deze aspecten komt uiteindelijk de "quality" feature. Deze feature legt vast wat de kwaliteit van een specifieke soort rode wijn is. De dataset bestaat uit 1600 records, welke genoeg zullen zijn om te gebruiken voor de train- en testdata. Deze dataset is [hier](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009?resource=download) terug te vinden.

### Imports en inlezen data
In de onderstaande code is te zien hoe de dataset wordt ingelezen en welke libraries worden gebruikt.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix as cm
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import preprocessing
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from scipy.special import expit

df = pd.read_csv('winequality-red.csv')


### Preperatie voor Logistic Regression en Classification
In de onderstaande code is te zien hoe een label wordt aangemaakt wat gebruikt kan worden voor classificatie en logistic regression. Dit wordt gedaan door middel van de "quality" feature te doorlezen en sorteren op hoge kwaliteit en lage kwaliteit. Dit wordt opgeslagen als een nieuwe feature, namelijk de "quality_range" feature. Aangezien het alleen de waardes 1 en 0 kan hebben, is het geschikt voor classificatie.

In [ ]:
# Low en High quality wine set (0 for low, 1 for high), set in label 'quality_range'
bins = [0, 5, 10]

labels = [0, 1]

df_bins = df.copy()

df_bins['quality_range'] = pd.cut(x=df_bins['quality'], bins=bins, labels=labels)


## Uitwerkingen


### Multiple Linear Regression
Multiple Linear Regression is een statische methode die gebruikt wordt om de relatie tussen meerdere onafhankelijke variabelen en een afhankelijke variabele. Het is een verlenging van linear regression, waar maar een onafhankelijke variabele en maar een afhankelijke variabele aanwezig is. De relaties tussen de variabelen worden met behulp van een lineare formule. De formule heeft de volgende vorm:

Y = b0 + b1X1 + b2X2 + ... + bn*Xn

Waar Y de afhankelijke variabel is. X1, X2, ..., Xn zijn de onafhankelijke variabelen. b0, b1, b2, ...., bn zijn de coëfficiënten die gekoppeld zijn aan de variabelen. De coëfficiënten representeren de relatie tussen iedere onafhankelijke variabele en de afhankelijke variabele. 

#### Voorbeeld
In het geval van onze code, wordt multiple linear regression toegepast op onze gevonden dataset. Hierbij is de feature "quality" de afhankelijke variabele en zijn de rest van de features de onafhankelijke variabelen. Hierdoor worden alle coëfficiënten berekend, waarmee de formule kan worden opgesteld.

In de onderstaande code is te zien hoe allereerst de afhankelijke variabelen en onafhankelijke variabelen gesplitst worden. Daarnaast is te zien hoe er een train- en testdataset opgesteld wordt. 

Hierna is het zichtbaar hoe de multiple linear regression uitgewerkt wordt in Sklearn. Er wordt een model opgesteld wat getraind wordt met de traindata. Hierna kan gekeken worden naar hoe goed het model presteerd door verwachtingen uit te voeren over te testdata.

Daarnaast worden hier de intercept en de coëfficiënten geprint die berekend zijn. Hiermee kan de formule opgesteld worden.

In [ ]:
#Multiple Linear Regression
y_lin = df['quality']
X_lin = df.drop('quality',axis=1)

X_lin_train, X_lin_test, y_lin_train, y_lin_test = train_test_split(X_lin, y_lin, test_size=0.1, random_state=40)

#Sklearn solution
model_lin = LinearRegression()

model_lin.fit(X_lin_train, y_lin_train)

predictions = model_lin.predict(X_lin_test)

print("Intercept: \n", model_lin.intercept_)
print("Coefficients: \n", model_lin.coef_)


Hieronder is ook een uitwerking gemaakt met behulp van statsmodel.api. Deze werd ook uitgewerkt omdat het mogelijk was om een duidelijker overzicht terug te geven van de coëfficiënten en intercept.

In [ ]:
#SM (statsmodel.api) solution
X_lin_train = sm.add_constant(X_lin_train)

model_sm = sm.OLS(y_lin_train, X_lin_train).fit()
predictions_sm = model_sm.predict(X_lin_train)

print_model = model_sm.summary()
print(print_model)

In het onderstaande stuk code wordt de accuraatheid van het model berekend. Daarnaast worden de R2-score, Mean Squared Error en Root Mean Squared Error berekend.

In [ ]:
#Accuracy Testing
train_accuracy = model_lin.score(X_lin_train, y_lin_train)
test_accuracy = model_lin.score(X_lin_test, y_lin_test)

y_lin_prediction = model_lin.predict(X_lin_test)

lin_score = r2_score(y_lin_test, y_lin_prediction)

print('One-vs-rest', '-'*35, 
      'Accuracy in Train Group   : {:.2f}'.format(train_accuracy), 
      'Accuracy in Test  Group   : {:.2f}'.format(test_accuracy), sep='\n')

print("\n")

print('R2 and Mean errors', '-'*35, 
      'R2-score   : {:.2f}'.format(lin_score), 
      'Mean Squared Error   : {:.2f}'.format(mean_squared_error(y_lin_test,y_lin_prediction)),
      'Root Mean Squared Error   : {:.2f}'.format(np.sqrt(mean_squared_error(y_lin_test,y_lin_prediction))), sep='\n')


### Logistic Regression
Logistic Regression is een methode voor binaire classificatie dat gebruikt maakt van een logistieke formule om de kans op een mogelijkheid te voorspellen. De Logistieke formule neemt een input van 1 of meerdere onafhankelijke variabelen en koppelt dit aan een getal tussen 0 en 1. Dit representeert de kans op deze waarde. De coëfficiënten van de onafhankelijke variabelen zijn opgehaald via trainingsdata. Het model wat hiermee is opgesteld wordt getraint om de optimale coëfficiënten te gebruiken om een binaire waarde te voorspellen. Zodra het model getraind is kan het gebruikt worden om voorspellingen over nieuwe data uit te voeren. 

De formule voor Logistic Regression heeft de volgende vorm:

P = 1 / (1 + e^-(b0 + b1X1 + b2X2 + ... + bn*Xn))

Waar P de kans is dat afhankelijke variabele 1 is. X1, X2, ..., Xn zijn de onafhankelijke variabelen. b0, b1, b2, ...., bn zijn de coëfficiënten die gekoppeld zijn aan de variabelen. De coëfficiënten representeren de relatie tussen iedere onafhankelijke variabele en de afhankelijke variabele. e is de wiskunde constante die ongeveer 2,718 is.

#### Voorbeeld
In het geval van onze code, wordt logistic regression toegepast op onze gevonden dataset. Hierbij is de feature "quality_range" de afhankelijke variabele en zijn de rest van de features de onafhankelijke variabelen. Hierdoor worden alle coëfficiënten berekend, waarmee de formule kan worden opgesteld.

Hieronder is te zien hoe de binaire afhankelijke variabele gesplitst wordt van de onafhankelijke variabelen. Daarnaast wordt hier de data opgesplitst in de train- en testset.

In [ ]:

#Set X and y, also split into train and test sets
X_log = df_bins[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']]

y_log = df_bins['quality_range']

X_log_train, X_log_test, y_log_train, y_log_test = train_test_split(X_log, y_log, test_size=0.1, random_state=40)

Hieronder is te zien hoe de logistic regression wordt uitgevoerd met behulp van Sklearn. Hier wordt gelijk de accuraatheid bepaald met de train- en testset.

In [ ]:
#Logistic regression
model_lr = LogisticRegression(random_state=40, max_iter=1600)

model_lr.fit(X_log_train, y_log_train)

train_accuracy = model_lr.score(X_log_train, y_log_train)
test_accuracy = model_lr.score(X_log_test, y_log_test)

print('One-vs-rest', '-'*35, 
      'Accuracy in Train Group   : {:.2f}'.format(train_accuracy), 
      'Accuracy in Test  Group   : {:.2f}'.format(test_accuracy), sep='\n')


In het onderstaande stuk code is te zien hoe er een matrix wordt opgesteld om de accuraatheid van het model weer te geven. Hier is te zien hoe accuraat het model voorspeld en hoe het model voorspeld.

In [ ]:

log_predictions = model_lr.predict(X_log_test)

#confusion matrix
score = round(accuracy_score(y_log_test, log_predictions), 3)
confusionMatrix_test = cm(y_log_test, log_predictions)
sns.heatmap(confusionMatrix_test, annot=True, fmt='.0f')
plt.xlabel('Predicted Values')
plt.ylabel('Actual Values')
plt.title('Accuracy Score: {0}'.format(score), size = 15)
plt.show()

#Array confusion matrix
log_pred_test = model_lr.predict(X_log_test)
log_pred_train = model_lr.predict(X_log_train)

cm = cm(y_log_test, log_pred_test)
print(cm)

In het onderstaande stuk code wordt de ROC-curve berekend van de voorspellingen van het model.

In [ ]:
#ROC-curve

probs = model_lr.predict_proba(X_log_test)[:,1] #predifct probabilities for the test data

fpr, tpr, thresholds =  roc_curve(y_log_test, probs) #Get ROC Curve

plt.figure(figsize=(8,5))
#Plot ROC curve
plt.plot([0,1], [0,1], 'k--')
plt.plot(fpr, tpr)
plt.xlabel('1 - Specificity Score')
plt.ylabel('Recall Score')
plt.title('ROC-Curve')
plt.show()


## Conclusie
### Multiple Linear Regression vs Logistic Regression
Logistic regression bleek uit deze opdracht accurater te zijn in het voorspellen van de kwaliteit van de rode wijnen. Dit komt doordat de kwaliteit van wijnen een balans van meerdere aspecten is. Het toevoegen of verminderen van een bepaald aspect betekend niet altijd dat de kwaliteit lineair veranderd. Dit is terug te zijn in de accuraatheidscores in beide modellen. 